#Considerações

1. Tensão de cisalhamento
==========================

##$\tau = \frac{FD}{2}$

##$\tau = \frac{8FD}{\pi d^3} + \frac{4F}{\pi d^2}$

##$\tau = K_s\frac{8FD}{\pi d^3}$

$K_s$ é um fator de correção da tensão de cisalhamento.
Usando o fator Bergsträsser, temos que:

##$\tau = K_B\frac{8FD}{\pi d^3}$
Onde,

$K_B = \frac{4C + 2}{4C - 3}$;

$C = \frac{D}{d}$, d = diâmetro do fio tabelado;

$4\leq C \leq 12$.


2. Ponto de operação
======================

O percurso fracionário até o fechamento da mola deve ser contido aos 75%.

##$F_s = (1 + \xi)F_{max}$
Onde
##$\xi  \geq 0.15$
Portanto, a tensão de cisalhamento:
##$\tau_s = K_B\frac{8(1 + \xi)F_{max}D}{\pi d^3}$

3. A cargo do projetista
========================

+ Escolher diâmetro $d$ do fio na tabela
+ O diâmetro médio “D” pode ser obtido a partir de $C$
+ Fator Bergsträsser $K_B$ é obtido a partir de $C$
+ $F_{max}$ ou $F$ são conhecidos
+ **Índice $C$ da mola**
+ **$\xi \geq 0.15$ deve ser seguido**

#Resistência a Compressão

Resistência a tração:

##$S_{ut} = \frac{A}{d^m}$

$S_{ut}$: Resistência à tração

𝐴 : Constante (obtida na Tabela)

𝑚: expoente (obtido na Tabela)

Resistência ao escoamento por cisalhamento:

##$S_{sy} = x * S_{ut}$

$x$: Constante da Tabela Porcentagem Máxima de resistência de Tração

1. Coeficiente de segurança
===========================

##$n_s = \frac{S_{sy}}{\tau_s}$

+ **$n_s$: Coeficiente de segurança deve ser $\geq 1.2$**

#Considerações Adicionais (opcional)

1. Rigidez da mola:

##$k = \frac{F}{y}$
e,

##$N_a = \frac{d^4G}{8D^3k}$

onde,

**$N_a$: Número de espiras ativas, deve respeitar $3 \leq N_a \leq 15$.**

$G$: Módulo de elasticidade transversal

In [ ]:
import numpy as np
import tkinter as tk
from tkinter import ttk

############################################# Classe Material
class Material:
    def __init__(self, nome, diametros_fio, expoente_m, const_material,
                 percent_max_Sut, elasticity):
        """
        Inicializa um novo objeto Material.
        :param id: identificação do material
        :param diametros_fio: Lista de valores de diâmetros de fio em mm.
        :param expoente_m: Valor do expoente m.
        :param A: Valor de A em MPa*mm**m.
        :param x: Porcentagem máxima de resistência de tração.
        :param G: módulo de elasticidade transversal
        """
        self.id = nome
        self.diametros_fio = diametros_fio
        self.m = expoente_m
        self.A = const_material # Constante material
        self.x = percent_max_Sut
        self.elasticity = elasticity
        self.G = 0

    def __str__(self):
        return (f"Material {self.id} com diâmetros: {self.diametros_fio}, "
                f"m: {self.m}, A: {self.A}, "
                f"Porcentagem Máxima de Resistência de Tração: {self.x * 100}%")
    
    def getElasticity(self, value):
        intervals = self.getElasticity["intervals"]
        values = self.elasticity["values"]

        for i, interval in enumerate(intervals):
            if interval[0] <= value < interval[1]:
                self.G = values[i]
                return self.G
    
def getMaterialById(id, materialList):
    for mat in materialList:
        if mat.id == id:
            return mat
    return None

# Materiais disponíveis
material_A227 = Material(
    nome="A227",
    diametros_fio = [
    0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.4, 1.6, 1.8, 2.0,
    2.2, 2.5, 2.8, 3.0, 3.5, 4.0, 4.5, 5.0, 5.5, 6.0,
    6.5, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0
    ],
    expoente_m=0.190,
    const_material=1783,
    percent_max_Sut=45/100,
    elasticity = {
                  "intervals":[[0.0, 0.8], [0.8, 1.61], [1.61, 3], [3, 999]],
                  "values":[82.7, 81.7, 81.0, 80]
                 }
)

material_A228 = Material(
    nome="A228",
    diametros_fio = [
    0.1, 0.12, 0.16, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5,
    0.55, 0.6, 0.65, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.4,
    1.6, 1.8, 2.0, 2.2, 2.5, 2.8, 3.0, 3.5, 4.0, 4.5, 
    5.0, 5.5, 6.0, 6.5
    ],
    expoente_m=0.145,
    const_material=2211,
    percent_max_Sut=45/100,
    elasticity = {
                  "intervals":[[0.0, 0.8], [0.8, 1.61], [1.61, 3], [3, 999]],
                  "values":[80.7, 80.0, 79.3, 78.6]
                 }
)

available_materials = [material_A227, material_A228]

############################################# Classe Mola
class Mola:
    def __init__(self):
        """
        Inicializa um novo objeto Mola.
        :param material: objeto material
        :param d: diâmetro do fio em mm.
        :param D: diâmetro da mola em mm.
        :param C: Índice da mola.
        :param force: força atuante na mola.
        :param n_s: coeficiente de segurança.
        """
         
        self.material = None
        self.d = -1
        self.D = -1
        self.C = -1
        self.force = None
        self.n_s = 0

        self.k = 0
        self.N_a = 0
        self.G = 0

        self.__xi = 0.15
        self.__lockedSafetyFactor = 1.2

    @property
    def xi(self):
        return self.__xi
    
    @xi.setter
    def xi(self, value):
        if value >= 0.15:
            self.__xi = value

    @property
    def lockedSafetyFactor(self):
        return self.__lockedSafetyFactor
    
    @lockedSafetyFactor.setter
    def lockedSafetyFactor(self, value):
        if value >= 1.2:
            self.__lockedSafetyFactor = value

    def getActiveSpires(self):
        self.N_a = self.d**4 * self.G / (8* self.D**3 * self.k)

    def update_spring(self):
        if self.d > 0 and self.C > 0:
            self.D = self.getSpringDiam()
            print(f"Diâmetro da mola (D): {self.D} mm")
            self.perform_calculations()
        elif self.d > 0 and self.force is not None:
            self.suggestC()

    def perform_calculations(self):
        if self.D > 0 and self.force is not None:
            K_B = self.getBergtrasserFactor()
            shearStress = self.getShearStress(self.force)
            tensileStrength = self.getTensileStrength()
            shearYieldStrength = self.getShearYieldStrength()
            safetyFactor = self.getSafetyFactor()
            
            response = {"K_B": K_B,
                        "T_s": shearStress,
                        "S_ut": tensileStrength,
                        "S_sy": shearYieldStrength,
                        "n_s": safetyFactor}
            
            print(f"Bergtrasser: {K_B}")
            print(f"Shear Stress: {shearStress} MPa")
            print(f"Tensile Strength: {tensileStrength} MPa")
            print(f"Shear Yield Strength: {shearYieldStrength} MPa")
            print(f"Safety Factor: {safetyFactor}")
            self.suggestC()

            return response

    def getSpringDiam(self):
        """
        Calcula e retorna o diâmetro da mola.
        """
        return self.C * self.d

    def getBergtrasserFactor(self):
        """
        Calcula e retorna o fator de correção de Bergstrasser (K_B).
        """
        return (4 * self.C + 2) / (4 * self.C - 3)

    def getShearStress(self, Forca):
        """
        Calcula e retorna a tensão de cisalhamento na mola.
        :param Forca: A força aplicada na mola.
        T_s = K8(1 + xi)FD/(pi d^3)
        """
        K = self.getBergtrasserFactor()
        return K * 8 * (1 + self.xi) *Forca * self.D / (np.pi * self.d ** 3)

    def getTensileStrength(self):
        """
        Calcula e retorna a resistência à tração do material.
        S_ut = A/d^3
        """
        return self.material.A / (self.d ** self.material.m)

    def getShearYieldStrength(self):
        """
        Calcula e retorna o limite de escoamento ao cisalhamento.
        S_sy = x * S_ut
        """
        TensileStrenght = self.getTensileStrength()
        return self.material.x * TensileStrenght 

    def getSafetyFactor(self):
        """
        Calcula e retorna o fator de segurança.
        :param ShearYS: Limite de escoamento ao cisalhamento.
        n_s = S_sy / T_s
        """
        ShearYieldStrength = self.getShearYieldStrength()
        shearStress = self.getShearStress(self.force)
        self.n_s = ShearYieldStrength / shearStress
        return self.n_s
    
    def suggestC(self):
        """
        Calcula o índice da mola.
        :param ShearYS: Limite de escoamento ao cisalhamento.
        n_s = S_sy / T_s
        """
        alpha = self.getShearYieldStrength() / self.lockedSafetyFactor
        beta = 8*(1+self.xi) * self.force / (np.pi * self.d**2)
        sqrt_arg = ((2* alpha - beta) / (4*beta))**2 - 3 *alpha / (4*beta)
        if sqrt_arg > 0:
            C = (2* alpha - beta) / (4*beta) + np.sqrt(sqrt_arg)
            print(f"Índice de mola sugerido: {C}")
        else:
            print("Carga incompatível com parâmetros da mola")

############################################# Classe App
class App:
    def __init__(self, root):
        self.root = root
        self.root.title('Projeto Slinky')

        # Criar o objeto mola
        self.mola = Mola()

        # Interface inicial
        self.create_initial_interface()

        # Interface de análise
        self.create_analysis_interface()

        # Interface de material
        self.create_material_interface()

    def create_initial_interface(self):
        self.inicial_widget = tk.Frame(self.root, bd=25)
        self.inicial_widget.grid(row=0, column=0, sticky='NSWE')

        # Dropdown menu para Materiais
        tk.Label(self.inicial_widget, text='Material:').grid(row=0, column=0, sticky='W')
        self.dropdown_material_var = tk.StringVar(value="")
        self.dropdown_materials_menu = ttk.Combobox(self.inicial_widget, textvariable=self.dropdown_material_var,
                                                    values=[mat.id for mat in available_materials])
        self.dropdown_materials_menu.bind("<<ComboboxSelected>>", self.on_dropdown_materials_select)
        self.dropdown_materials_menu.grid(row=0, column=1)

        # Dropdown menu para Diâmetros de Fio
        tk.Label(self.inicial_widget, text='Diâmetro [mm]:').grid(row=1, column=0, sticky='W')
        self.dropdown_diameter_var = tk.StringVar(value="")
        self.dropdown_diameters_menu = ttk.Combobox(self.inicial_widget, textvariable=self.dropdown_diameter_var)
        self.dropdown_diameters_menu.bind("<<ComboboxSelected>>", self.on_dropdown_diameter_select)
        self.dropdown_diameters_menu.grid(row=1, column=1, sticky='W')

        # Slider para Índice C
        tk.Label(self.inicial_widget, text='Índice C:').grid(row=2, column=0, sticky='W')
        self.slider_var = tk.DoubleVar(value=4.0)
        self.c_widget = tk.Frame(self.inicial_widget)
        self.c_widget.grid(row=2, column=1)
        self.slider = ttk.Scale(self.c_widget, from_=4.0, to=12.0, orient="horizontal", variable=self.slider_var,
                                command=self.on_slider_change)
        self.slider.grid(row=0, column=0, sticky='W')
        self.slidervalue_label = tk.Label(self.root, text=self.slider_var.get())
        self.C_entry = tk.Entry(self.c_widget, textvariable=self.slider_var, width=6)
        self.C_entry.grid(row=0, column=1, sticky='W')
        tk.Button(self.inicial_widget, text="Definir", command=self.on_C_entry).grid(row=2, column=2, sticky='W')

        # Caixa de entrada para Força
        tk.Button(self.inicial_widget, text="Força atuante (N):", command=self.on_number_entry).grid(row=3, column=0)
        self.number_entry = tk.Entry(self.inicial_widget)
        self.number_entry.grid(row=3, column=1, sticky='W')

    def create_analysis_interface(self):
        self.analise_widget = tk.Frame(self.root, bd=25)
        self.analise_widget.grid(row=4, column=0, sticky='NSWE')

        labels = ['Diâmetro da mola (mm):', 'Fator de Bergstrasser (K_B):',
                  'Tensão de Cisalhamento (kN):', 'Fator de segurança (n_s):']
        res_labels = ["D", "K_B", "T_s", "n_s"]

        self.analysis_labels = {}  # Dicionário para armazenar as labels

        for i, (label, res_label) in enumerate(zip(labels, res_labels)):
            tk.Label(self.analise_widget, text=label).grid(row=i, column=0, sticky='W')
            self.analysis_labels[res_label] = tk.Label(self.analise_widget, text=res_label,
                                                       bg='white', width=8)
            self.analysis_labels[res_label].grid(row=i, column=1, sticky='W')

    def create_material_interface(self):
        self.material_widget = tk.Frame(self.root, bd=25)
        self.material_widget.grid(row=4, column=1, sticky='NSWE')

        labels = ['Resistência à tração (kN):', 'Limite de escoamento ao cisalhamento (kN):']
        res_labels = ["S_ut", "S_sy"]

        self.material_labels = {}  # Dicionário para armazenar as labels

        for i, (label, res_label) in enumerate(zip(labels, res_labels)):
            tk.Label(self.material_widget, text=label).grid(row=i, column=0)
            self.material_labels[res_label] = tk.Label(self.material_widget, text=res_label,
                                                       bg='white', width=8)
            self.material_labels[res_label].grid(row=i, column=1, sticky='W')

    def on_dropdown_materials_select(self, event):
        selected_material_str = self.dropdown_material_var.get()
        self.mola.material = getMaterialById(selected_material_str, available_materials)
        new_diameters = self.mola.material.diametros_fio
        self.dropdown_diameters_menu['values'] = new_diameters
        self.dropdown_diameter_var.set('')

    def on_dropdown_diameter_select(self, event):
        self.mola.d = float(self.dropdown_diameter_var.get())
        self.mola.update_spring()

    def on_slider_change(self, value):
        self.mola.C = self.slider_var.get()
        self.slidervalue_label.config(text=f"{self.slider_var.get():.2f}")
        self.mola.update_spring()

    def on_number_entry(self):
        try:
            self.mola.force = float(self.number_entry.get())
            self.mola.update_spring()
            values = self.mola.perform_calculations()
            self.update_analysis_labels(values)
        except ValueError:
            print("Número inválido inserido para a força.")

    def on_C_entry(self):
        try:
            value = float(self.C_entry.get())
            if 4 <= value <= 12:
                self.slider.set(value)
                self.mola.C = value
                self.mola.update_spring()

        except ValueError:
            pass

    def update_analysis_labels(self, vals):
        self.analysis_labels['D'].config(text=f"{self.mola.D:.2f}")
        self.analysis_labels['K_B'].config(text=f"{vals["K_B"]:.2f}")
        self.analysis_labels['T_s'].config(text=f"{vals["T_s"]:.2f}")
        self.analysis_labels['n_s'].config(text=f"{vals["n_s"]:.2f}")

        self.material_labels['S_ut'].config(text=f"{vals["S_ut"]:.2f}")
        self.material_labels['S_sy'].config(text=f"{vals["S_sy"]:.2f}")

def main():
    # Criar a janela principal
    root = tk.Tk()
    app = App(root)    
    root.mainloop()

if __name__ == "__main__":
    main()